In [14]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist

In [15]:
def _numerical_gradient_1d(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        
    return grad


def numerical_gradient_2d(f, X):
    if X.ndim == 1:
        return _numerical_gradient_1d(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_1d(f, x)
        
        return grad


def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

In [16]:

def identity_function(x):
    return x


def step_function(x):
    return np.array(x > 0, dtype=np.int)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))    


def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)
    

def relu(x):
    return np.maximum(0, x)


def relu_grad(x):
    grad = np.zeros(x)
    grad[x>=0] = 1
    return grad
    

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))


def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


def softmax_loss(X, t):
    y = softmax(X)
    return cross_entropy_error(y, t)

In [17]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화 
        self.params = {}
        self.params['w1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        
        self.params['b1'] = np.zeros(hidden_size)
        self.params['w2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        w1, w2 = self.params['w1'], self.params['w2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, w1) + b1
        z1 = sigmoid(a1) 
        
        a2 = np.dot(z1, w2) + b2
        y = softmax(a2)
        
        return y
    
    # x: 입력데이터 t: 정답레이블 
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    # 정확도 검사 
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # 
    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x,t)
        
        grads = {}
        grads['w1'] = numerical_gradient_2d(loss_w, self.params['w1'])
        grads['b1'] = numerical_gradient_2d(loss_w, self.params['b1'])
        grads['w2'] = numerical_gradient_2d(loss_w, self.params['w2'])
        grads['b2'] = numerical_gradient_2d(loss_w, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        
        w1 = self.params['w1']
        b1 = self.params['b1']
        
        w2 = self.params['w2']
        b2 = self.params['b2']
        
        batch_size = x.shape[0] #입력 데이터 수
        
        #grads 딕셔너리 객체 생성 
        grads = {}
        
        a1 = np.dot(x, w1) +b1
        z1 = sigmoid(a1)
        
        a2 = np.dot(z1, w2) + b2
        y = softmax(a2)
        
        da2 = (y-t) / batch_size
        
        grads['w2'] = np.dot(z1.T, da2)
        grads['b2'] = np.sum(da2, axis=0)
        
        dz1 = np.dot(da2, w2.T)
        da1 = ((1-z1)*z1) * dz1
        
        grads['w1'] = np.dot(x.T, da1)
        grads['b1'] = np.sum(da1, axis=0)
        
        
        return grads

In [18]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, 
                                                 one_hot_label=True)
train_loss_list=[]

In [19]:
# 하이퍼파라미터 (사용자가 지정해줘야하는 변수)
iters_num =10000 # 반복횟수 
train_size = x_train.shape[0] # 데이터 개수 shape[0] , 행의개수
batch_size = 100 #미니배치 크기 
learning_rate = 0.1 #학습률
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10) # 은닉층 적절히 임의로


In [20]:
for i in range(iters_num):
    # 미니배치 획득 
    
    #훈련데이터 개수에서, batch_size만큼 랜덤선택
    #인덱스만 선택이 된다
    batch_mask = np.random.choice(train_size, batch_size) 
    
    #그 인덱스들을 넣어서 임의의 x, t 데이터 선택 
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산 
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # 매개변수 갱신 
    for key in ('w1','b1','w2','b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # 학습 경과 기록 
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

KeyboardInterrupt: 